In [66]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.schema.runnable import RunnablePassthrough


from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")


llm = ChatOpenAI(openai_api_key=api_key,
                  temperature=0.1,
                  streaming=True,
                  )
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True
) 

examples = [
    {
        "question": "탑건",
        "answer":"🛩️👨‍✈️🔥",
    },
    {
        "question": "대부",
        "answer": "👨‍👨‍👦🔫🍝",
    },
]
example_prompt = ChatPromptTemplate.from_messages([
    ("human",  "{question}",),  
    ("ai", "{answer}")
])

examples_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화이름을 받고, 그 영화 줄거리를 기반으로 3 개의 이모티콘으로만 응답합니다. 만약, 어떤 영화인지 모를 경우에는 물음표 이모티콘을 3개로 응답합니다."),
    examples_prompt,
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}",)
    
])

def load_memory(_):
        return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(question):
        result = chain.invoke({"question": question})
        memory.save_context(
                {"input":question},
                {"output":result.content}
        )
        print(result)

invoke_chain('아바타')

content='❄️🌿🌌'


In [67]:

invoke_chain('라푼젤')

content='👸🏼✨🦎'


In [68]:
invoke_chain('겨울왕국')

content='❄️👑⛄️'


In [69]:
invoke_chain('곡성')

content='❓❓❓'


In [70]:
invoke_chain('인터스텔라')

content='🚀⏳🌌'


In [71]:
invoke_chain('가필드 더 무비')

content='❓❓❓'


In [72]:
invoke_chain('the garfield movie')

content='🐱🍕🎬'


In [73]:
invoke_chain('겨울왕국2')

content='👸🏼❄️⛄️'


In [74]:
invoke_chain('슈퍼배드')

content='👾😈💣'
